## Королев Федор ПИ15. Домашнее задание №2



## Часть 1. ML workflow (**всего 5 баллов**)

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_squared_error, r2_score

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

from itertools import product

### Загрузим данные для работы.

In [2]:
df = pd.read_csv("winequality-red.csv")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


Будем решать задачу регрессии: необходимо предсказать качество вина на основе его характеристик

### Шаг 1.  (**0.2 балла**)
Создайте матрицу X объект-признак и целевой вектор y ("quality")

То есть матрица X будет содержать в себе все признаки, кроме "quality". Целевой вектор y будет состоять из столлбца "quality"

In [3]:
# Матрица признаков X: все столбцы, кроме "quality"
X = df.drop("quality", axis=1)

# Целевой вектор y: столбец "quality"
y = df["quality"]

# Проверим размерности матрицы X и вектора y
X.shape, y.shape

((1599, 11), (1599,))

Получаем 1599 строк на 11 столбцов в X и 1599 строк столбце y

### Шаг 2. (**0.2 балла**)
Разбейте данные на train и test (доля тестовых данных - 30%).

Получается, что данные необходимо разбить на две части:

- треннировочные данные (train): 70% данных, на которых модель будет обучаться
- тестовые данные (test): 30% данных, на которых мы будем проверять, насколько хорошо модель предсказывает качество вина

Для этого используем функцию train_test_split из библиотеки sklearn

In [4]:
# Разделение данных на тренировочные и тестовые данные
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Проверим размерности тренировочных и тестовых данных
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((1119, 11), (480, 11), (1119,), (480,))

### Шаг 3. (**0.2 балла**)
Обучите линейную регрессию на тренировочных данных и сделайте предсказания на train и на test.

Для более наглядного вывода результатов предсказаний я использовал форматированный вывод: вывел несколько первых предсказанных значений для тренировочной и тестовой выборок, а после сравнил их с реальными значениями

In [5]:
# Инициализация модели линейной регрессии
linear_reg = LinearRegression()

# Обучение модели на тренировочных данных
linear_reg.fit(X_train, y_train)

# Предсказания на тренировочных данных
y_train_pred = linear_reg.predict(X_train)

# Предсказания на тестовых данных
y_test_pred = linear_reg.predict(X_test)


# Проверка нескольких предсказанных значений для обоих наборов

# Для тренировочной выборки
train_results = pd.DataFrame({
    'Real Quality (Train)': y_train.values,        # Вывод обучающих реальных значений
    'Predicted Quality (Train)': y_train_pred      # Вывод обучающих предсказанных значений
}).head()

# Для тестовой выборки
test_results = pd.DataFrame({
    'Real Quality (Test)': y_test.values,          # Вывод тестовых реальных значений
    'Predicted Quality (Test)': y_test_pred        # Вывод тестовых предсказанных значений
}).head()

print("Результаты для тренировочной выборки:\n", train_results)
print("\nРезультаты для тестовой выборки:\n", test_results)


Результаты для тренировочной выборки:
    Real Quality (Train)  Predicted Quality (Train)
0                     7                   6.358827
1                     5                   5.977426
2                     5                   5.668873
3                     6                   5.672820
4                     5                   5.388390

Результаты для тестовой выборки:
    Real Quality (Test)  Predicted Quality (Test)
0                    6                  5.356763
1                    5                  5.090715
2                    6                  5.625538
3                    5                  5.448861
4                    6                  5.744784


### Шаг 4. (**0.4 балла**)
Выведите на экран ошибку MSE на train и на test, затем выведите на экран ошибку r2 на train и test.  

Сначала рассчитаем среднеквадратическую ошибку (MSE; чем меньше, тем лучше), затем вычислим коэффициент детерминации (R^2; показывает, какая доля дисперсии целевой переменной объясняется моделью; чем ближе к 1, тем лучше)

In [6]:
# Рассчёт MSE для тренировочных и тестовых данных
mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

# Рассчёт R^2 для тренировочных и тестовых данных
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

# Шаг 3: Вывод результатов
print(f"Среднеквадратическая ошибка (MSE) на тренировочных данных: {mse_train:.2f}")
print(f"Среднеквадратическая ошибка (MSE) на тестовых данных: {mse_test:.2f}")

print(f"Коэффициент детерминации (R^2) на тренировочных данных: {r2_train:.2f}")
print(f"Коэффициент детерминации (R^2) на тестовых данных: {r2_test:.2f}")

Среднеквадратическая ошибка (MSE) на тренировочных данных: 0.42
Среднеквадратическая ошибка (MSE) на тестовых данных: 0.41
Коэффициент детерминации (R^2) на тренировочных данных: 0.36
Коэффициент детерминации (R^2) на тестовых данных: 0.35


### Шаг 5. (**0.5 балла**)
Вычислите среднее качество (r2) модели на кросс-валидации с k=5 фолдами.

Кросс-валидация разбивает данные на 5 частей (фолдов), затем обучает модель на 4 из них, а на оставшейся - тестиуется, затем выбирается новый фолд для тестирования и так всего 5 раз. В итоге получаем 5 оценок качества модели

In [7]:
# Проведение кросс-валидации для модели линейной регрессии
r2_scores = cross_val_score(linear_reg, X, y, cv=5, scoring='r2')

# Вывод результатов кросс-валидации
print(f"R^2 на каждом фолде: {r2_scores}")
print(f"Средний R^2 на кросс-валидации (k=5): {r2_scores.mean():.2f}")


R^2 на каждом фолде: [0.13200871 0.31858135 0.34955348 0.369145   0.2809196 ]
Средний R^2 на кросс-валидации (k=5): 0.29


### Шаг 6.  (**0.5 балла**)
Теперь примените линейную регрессию с L1-регуляризацией (Lasso) для данной задачи. Объявите модель и подберите параметр регуляризации alpha по сетке. Ищите alpha в диапазоне (0.1, 1.1) с шагом 0.1.

Осуществите подбор параметра alpha по тренировочным данным (Xtrain, ytrain).

Метод GridSearchCV перебирает все возможные комбинации параметров из сетки значений и оценивает каждую из них с помощью кросс-валидации. В итоге выбирается лучший вариант

In [8]:
from sklearn.linear_model import Lasso

# Определяем сетку значений alpha от 0.1 до 1.1 с шагом 0.1
alpha_values = np.arange(0.1, 1.2, 0.1)
param_grid = {'alpha': alpha_values}

# Инициализация модели Lasso
lasso = Lasso()

# Инициализация GridSearchCV для подбора параметра alpha
grid_search = GridSearchCV(estimator=lasso, param_grid=param_grid, scoring='r2', cv=5)

# Обучение модели на тренировочных данных
grid_search.fit(X_train, y_train)

# Получение оптимального параметра alpha
best_alpha = grid_search.best_params_['alpha']

print(f"Оптимальное значение alpha: {best_alpha:.2f}")

Оптимальное значение alpha: 0.10


### Шаг 7.  (**0.5 балла**)
Выведите наилучший алгоритм и наилучшее качество по результатам подбора alpha (best_estimator_ и best_score_).

In [9]:
# Получение наилучшей модели
best_model = grid_search.best_estimator_

# Получение наилучшего качества (R^2) на кросс-валидации
best_r2_score = grid_search.best_score_

# Вывод результатов
print(f"Наилучшая модель: {best_model}")
print(f"Наилучшее качество (R^2) на кросс-валидации: {best_r2_score:.2f}")


Наилучшая модель: Lasso(alpha=0.1)
Наилучшее качество (R^2) на кросс-валидации: 0.24


### Шаг 8.  (**0.5 балла**)

С помощью найденного best_estimator_ сделайте предсказание на тестовых данных и выведите на экран r2-score на тесте.

In [10]:
# Предсказания на тестовых данных с помощью лучшей модели
y_test_pred_lasso = best_model.predict(X_test)

# Вычисление R^2 на тестовых данных
r2_test_lasso = r2_score(y_test, y_test_pred_lasso)

# Вывод результата
print(f"R^2 на тестовых данных с Lasso-регрессией: {r2_test_lasso:.2f}")


R^2 на тестовых данных с Lasso-регрессией: 0.21


### Шаг 9.  (**0.5 балла**)

Попробуем улучшить качество модели за счет добавления полиномиальных признаков. Создайте pipeline, состоящий из добавления полиномиальных признаков степени 2, а затем применения линейной регрессии.

Затем вычислите r2-score этой модели на кросс валидации с пятью фолдами.

Насколько я понял, полиномиальные признаки степени 2 - это новые признаки создаются путём добавления квадратичных и взаимодействующих комбинаций существующих признаков (то есть новые признаки будут включать квадраты старых и из произведения)

Pipeline в свою очередь автоматизирует процесс последовательного выполнения нескольких шагов, таких как предобработка данных и обучение модели

In [11]:
from sklearn.preprocessing import StandardScaler

# Создание Pipeline для полиномиальных признаков и линейной регрессии
pipeline = Pipeline([
    ('poly_features', PolynomialFeatures(degree=2)),  # Полиномиальные признаки степени 2
    ('linear_regression', LinearRegression())         # Линейная регрессия
])

# Кросс-валидация на 5 фолдах
r2_scores_poly = cross_val_score(pipeline, X, y, cv=5, scoring='r2')

# Вывод результатов
print(f"R^2 на каждом фолде с полиномиальными признаками: {r2_scores_poly}")
print(f"Средний R^2 на кросс-валидации с полиномиальными признаками: {r2_scores_poly.mean():.2f}")

R^2 на каждом фолде с полиномиальными признаками: [-0.03990649  0.29930017  0.38080623  0.30145898  0.20882197]
Средний R^2 на кросс-валидации с полиномиальными признаками: 0.23


### Шаг 10.  (**0.5 балла**)
Обучите модель (pipeline) на тренировочных данных и сделайте предсказания для train и test, затем выведите на экран r2-score и MSE на тренировочных и на тестовых данных.

In [12]:
# Обучение модели на тренировочных данных
pipeline.fit(X_train, y_train)

# Предсказания на тренировочных данных
y_train_pred_poly = pipeline.predict(X_train)

# Предсказания на тестовых данных
y_test_pred_poly = pipeline.predict(X_test)

# Вычисление метрик

# MSE (среднеквадратическая ошибка) для тренировочных и тестовых данных
mse_train_poly = mean_squared_error(y_train, y_train_pred_poly)
mse_test_poly = mean_squared_error(y_test, y_test_pred_poly)

# R^2 (коэффициент детерминации) для тренировочных и тестовых данных
r2_train_poly = r2_score(y_train, y_train_pred_poly)
r2_test_poly = r2_score(y_test, y_test_pred_poly)

# Вывод результатов

print(f"R^2 на тренировочной выборке с полиномиальными признаками: {r2_train_poly:.2f}")
print(f"MSE на тренировочной выборке: {mse_train_poly:.2f}")

print(f"R^2 на тестовой выборке с полиномиальными признаками: {r2_test_poly:.2f}")
print(f"MSE на тестовой выборке: {mse_test_poly:.2f}")


R^2 на тренировочной выборке с полиномиальными признаками: -0.12
MSE на тренировочной выборке: 0.74
R^2 на тестовой выборке с полиномиальными признаками: -0.22
MSE на тестовой выборке: 0.77


### Сделайте выводы. Для этого ответьте на вопросы: (**1 балл**)

1) Хорошее ли качество показала исходная модель (линейная регрессия без регуляризации)? Является ли эта модель переобученной?

2) Помогла ли L1-регуляризация улучшить качество модели?

3) Помогло ли добавление полиномов второй степени улучшить качество модели? Как добавление новых признаков повлияло на переобучение?

In [13]:
# 1) Хорошее ли качество показала исходная модель (линейная регрессия без регуляризации)?

"""
Дляя ответа на данный вопрос нужно посмотреть на результаты 3, 4 и 5 шагов:
Качество модели:

R^2 на тренировочных данных: 0.36
R^2 на тестовых данных: 0.35
Средний R^2 на кросс-валидации: 0.29

Из полученных результатов можно сделать вывод, что модель показала не самое высокое качество.
Оценка, как на тренировочных, так и на тестовых данных, достаточно низкая.
Это говорит о том, что модель объясняет лишь небольшую часть дисперсии целевого признака
"""
# Является ли эта модель переобученной?
"""
Разница между результатами тестов незначительна,оценка кросс-валидации не далеко ушла. Следовательно, модель не переобучена
"""

# 2) Помогла ли L1-регуляризация улучшить качество модели?

"""
Дляя ответа на данный вопрос нужно посмотреть на результаты 7 и 8 шагов:
Качество модели с Lasso-регуляризацией:

Лучший результат R^2 на кросс-валидации: 0.24
R^2 на тестовых данных: 0.21

L1-регуляризация с alpha=0.1 не улучшила качество модели, а лишь ухудшила результат.
Можно сделать вывод, что для данной задачи L1-регуляризация оказалась слишком сильной, что негативно сказалось на качестве модели
"""

# 3) Помогло ли добавление полиномов второй степени улучшить качество модели?

"""
Дляя ответа на данный вопрос нужно посмотреть на результаты 9 и 10 шагов:
Качество модели с полиномиальными признаками:

Средний R^2 на кросс-валидации: 0.23
R^2 на тренировочных данных: -0.12
R^2 на тестовых данных: -0.22

Добавление полиномиальных признаков не улучшило качество модели.
Результаты ккак на тренировочных, так и на тестовых данных стали отрицательными.
Это указывает на то, что модель даже хуже простой средней предсказательной модели
"""

# Как добавление новых признаков повлияло на переобучение?

"""
Значительное ухудшение качества на тренировочных данных (с отрицательным R^2) и ещё более низкие результаты на тесте говорят о том, что модель сильно переобучена.
Это связано с тем, что полиномиальные признаки увеличивают сложность модели, и вместо выявления закономерностей, она начинает как бы запоминать данные
"""

'\nЗначительное ухудшение качества на тренировочных данных (с отрицательным R^2) и ещё более низкие результаты на тесте говорят о том, что модель сильно переобучена.\nЭто связано с тем, что полиномиальные признаки увеличивают сложность модели, и вместо выявления закономерностей, она начинает как бы запоминать данные\n'

### *Попытайтесь улучшить модель (добейтесь наилучшего качества) - можно использовать любые методы.

При улучшении качества r2 на 0.1-0.2 +1 балл, при большем улучшении +2 балла (дополнительно к 5 баллам за основную часть).

Я решил использовать метод градиентного бустинга (методи, в котором несколько деревьев решений последовательно обучаются и исправляют ошибки предыдущих)

Основные шаги градиентного бустинга:

1. Первое дерево обучается на исходных данных.
2. Каждое следующее дерево обучается на ошибках предыдущих деревьев. Это означает, что оно пытается предсказать остатки (разницу между реальными значениями и предсказанными).
3. Градиентный спуск используется для нахождения оптимальных весов для следующего дерева с целью минимизации ошибки.
4. Процесс повторяется, и итоговый результат представляет собой взвешенную сумму всех деревьев.



In [14]:
from sklearn.ensemble import GradientBoostingRegressor

# Создадим Pipeline с нормализацией данных и моделью градиентного бустинга
pipeline_gb = Pipeline([
    ('scaler', StandardScaler()),                         # Нормализация данных
    ('gradient_boosting', GradientBoostingRegressor(random_state=42))  # Градиентный бустинг
])

# Зададим параметры для поиска через GridSearchCV
param_grid_gb = {
    'gradient_boosting__n_estimators': [100, 200, 300],   # Количество деревьев
    'gradient_boosting__learning_rate': [0.01, 0.1, 0.2], # Скорость обучения
    'gradient_boosting__max_depth': [3, 5, 7]             # Глубина (сложность) деревьев
}

# Используем GridSearchCV для подбора параметров
grid_search_gb = GridSearchCV(pipeline_gb, param_grid_gb, cv=5, scoring='r2', n_jobs=-1)

# Обучаем модель на тренировочных данных
grid_search_gb.fit(X_train, y_train)

# Выводим наилучшие параметры и результат на кросс-валидации
print(f"Наилучшие параметры: {grid_search_gb.best_params_}")
print(f"Лучший R^2 на кросс-валидации: {grid_search_gb.best_score_:.2f}")

# Делаем предсказания на тестовых данных
y_test_pred_gb = grid_search_gb.best_estimator_.predict(X_test)

# Вычисляем R^2 и MSE на тестовых данных
r2_test_gb = r2_score(y_test, y_test_pred_gb)
mse_test_gb = mean_squared_error(y_test, y_test_pred_gb)

print(f"R^2 на тестовых данных: {r2_test_gb:.2f}")
print(f"MSE на тестовых данных: {mse_test_gb:.2f}")

Наилучшие параметры: {'gradient_boosting__learning_rate': 0.01, 'gradient_boosting__max_depth': 7, 'gradient_boosting__n_estimators': 300}
Лучший R^2 на кросс-валидации: 0.42
R^2 на тестовых данных: 0.44
MSE на тестовых данных: 0.36


Исходная модель линейной регрессии имела средний R^2 на кросс-валидации 0.29. После градиентного бустинга показатель среднего R^2 на кросс-валидации выросло до 0.42.

Исходная линейная регрессия имела R^2 на тестовых данных 0.35. Градиентный бустинг показал R^2 0.44, что также является улучшением. MSE на тестовой выборке снизилось с 0.41 до 0.36, что подтверждает лучшее качество предсказаний

## Часть 2. Target encoding (**всего 5 баллов**)

В этом части домашнего задания вы будете работать с выборкой `1C`. Вам нужно посчитать счетчики для `item_id` четырьмя способами:

    1) При помощи KFold схемы;  
    2) При помощи Leave-one-out схемы;
    3) При помощи smoothing схемы;
    4) При помощи expanding mean схемы.

### Подготовка данных

In [15]:
sales = pd.read_csv('sales_train.csv.gz')
sales.columns = ['date', 'date_block_num', 'shop_id', 'item_id', 'item_price', 'target']
sales

,date,date_block_num,shop_id,item_id,item_price,target
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


In [16]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

# Для каждого месяца мы создаем сетку из всех магазинов/комбинаций товаров за этот месяц
grid = []
for block_num in sales['date_block_num'].unique():
    # Получаем уникальные идентификаторы магазинов для текущего месяца
    cur_shops = sales[sales['date_block_num'] == block_num]['shop_id'].unique()
    # Получаем уникальные идентификаторы товаров для текущего месяца
    cur_items = sales[sales['date_block_num'] == block_num]['item_id'].unique()
    # Создаем все возможные комбинации магазинов и товаров для текущего месяца и добавляем их в grid
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])), dtype='int32'))

# Объединяем все комбинации в один DataFrame с заданными индексами
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

# Группируем данные по магазинам, товарам и номерам блоков даты, суммируя целевую переменную
gb = sales.groupby(index_cols,as_index=False).agg({'target':'sum'})

# Объединяем сетку с агрегированными данными, заполняя отсутствующие значения нулями
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)

# Сортируем данные по номеру блока даты, идентификатору магазина и идентификатору товара
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

### Mean encodings без регуляризации

После проделанной технической работы, мы готовы посчитать счетчики для переменной `item_id`.

Ниже приведены две реализации подсчета счетчиков без регуляризации. Можно использовать данный код в качестве стартовой точки для реализации различных техник регуляризации.

#### Способ 1

In [17]:
# Вычисляем среднее значение целевой переменной для каждого уникального идентификатора товара
item_id_target_mean = all_data.groupby('item_id').target.mean()

# Создаем новую колонку 'item_target_enc' в DataFrame all_data, отображая средние значения целевой переменной для каждого товара
all_data['item_target_enc'] = all_data['item_id'].map(item_id_target_mean)

# Заполняем отсутствующие значения в 'item_target_enc' значением 0.3343
all_data['item_target_enc'].fillna(0.3343, inplace=True)

# Извлекаем значения из столбца 'item_target_enc' и сохраняем их в переменной encoded_feature
encoded_feature = all_data['item_target_enc'].values

# Вычисляем коэффициент корреляции между целевой переменной и закодированным признаком и выводим его на экран
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])


0.4830386988621699


<ipython-input-17-0a816b26c8f3>:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  all_data['item_target_enc'].fillna(0.3343, inplace=True)


#### Способ 2

In [18]:
'''
    В отличие от функции `.target.mean()`, функция `transform`
    вернет DataFrame с индексом, как в `all_data`.
    По сути, эта одна строка кода эквивалентна первым двум строкам из Метода 1.
'''
all_data['item_target_enc'] = all_data.groupby('item_id')['target'].transform('mean')

# Заполняем отсутствующие значения (NaN)
all_data['item_target_enc'].fillna(0.3343, inplace=True)

# Вычисляем и выводим коэффициент корреляции
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])


0.4830386988621699


<ipython-input-18-45b431b44915>:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  all_data['item_target_enc'].fillna(0.3343, inplace=True)


###  KFold схема (**1.25 балла**)

Необходимо реализовать Kfold схему с пятью фолдами. Используйте KFold(5) из sklearn.model_selection.

1. Разбейте данные на 5 фолдов при помощи `sklearn.model_selection.KFold` с параметром `shuffle=False`.
2. Проитерируйтесь по фолдам: используйте 4 обучающих фолда для подсчета средних значений таргета по `item_id` и заполните этими значениями валидационный фолд на каждой итерации.

Обратите внимание на **Способ 1** из примера. В частности, изучите, как работают функции map и pd.Series.map. Они довольно полезны во многих ситуациях.

KFold - это метод оценки моделей, в котором данные разбиваются на фолды и в каждой итерации один из фолдов используется для тестирования модели, а остальные - для обучения

In [19]:
from sklearn.model_selection import KFold

# Инициализация KFold для разбиения данных на 5 фолдов с параметром shuffle=False
kf = KFold(n_splits=5, shuffle=False)

# Создание массива для хранения закодированных признаков, который инициализируется нулями
encoded_feature = np.zeros(all_data.shape[0])

# Итерация по фолдам: разбиваем данные на обучающие и валидационные наборы
for train_index, val_index in kf.split(all_data):
    train_data = all_data.iloc[train_index]         # Получаем обучающий набор данных по индексам
    val_data = all_data.iloc[val_index]             # Получаем валидационный набор данных по индексам

    # Вычисляем средние значения целевой переменной для каждого item_id на обучающем наборе данных
    item_id_target_mean = train_data.groupby('item_id')['target'].mean()

    # Заполняем валидационный набор средними значениями по item_id
    # Используем метод map для сопоставления средних значений с item_id в валидационном наборе
    encoded_feature[val_index] = val_data['item_id'].map(item_id_target_mean).fillna(0.3343)

# Вычисление коэффициента корреляции между целевой переменной и закодированным признаком
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
# Выводим коэффициент корреляции на экран
print(corr)

0.4164590712798811


Ожидаемый ответ 0.4165

### Leave-one-out схема (**1.25 балла**)

Необходимо реализовать leave-one-out схему . Учтите, если вы запустите код из первого задания, задав количество фолдов такое же как размер выборки, то вы, вероятно, получите правильный ответ, но ждать будете очень-очень долго.

Для более быстрой реализации подсчета среднего таргета на всех объектах, кроме одного, вы можете:

1. Вычислить суммарный таргет по всем объектам.
2. Вычесть таргет конкретного объекта и разделить результат на `n_objects - 1`.

Заметим, что пункт `1.` следует сделать для всех объектов. Также заметим, что пункт `2.` может быть реализован без циклов `for`.

Здесь может оказаться полезной функция .transform из **Способа 2** из примера.

Leave-One-Out - это метод оценки, в котором количество фолдов равно количеству наблюдений в наборе данных (для каждого наблюдения в данных будет выполнена отдельная итерация, 1 наблюдение - тестовый набор, а остальные наблюдения - обучающий набор)

In [20]:
# Вычисляем сумму таргета по item_id
item_id_target_sum = all_data.groupby('item_id')['target'].transform('sum')

# Вычисляем количество объектов для каждого item_id
item_id_count = all_data.groupby('item_id')['target'].transform('count')

# Вычисляем закодированный признак, исключая текущий объект
# Используем item_id_count для корректного деления
encoded_feature = (item_id_target_sum - all_data['target']) / (item_id_count - 1)

# Заполняем NaN значениями, если они есть
encoded_feature.fillna(0.3343, inplace=True)

# Вычисляем коэффициент корреляции между целевой переменной и закодированным признаком
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
# Выводим коэффициент корреляции на экран
print(corr)

0.4803848311293002


Ожидаемый ответ 0.4803

### Smoothing (**1.25 балла**)

Необходимо реализовать smoothing с $\alpha = 100$. Используйте формулу:

$\frac{mean(target) \cdot nrows + globalmean \cdot \alpha }{nrows + \alpha}$,

где $globalmean=0.3343$. Заметим, что `nrows` - это количество объектов, принадлежащих конктертной категории, а не количество строк в датасете.

In [27]:
# Параметры для сглаживания
alpha = 100
globalmean = 0.3343

# Вычисляем среднее значение целевой переменной для каждого item_id
item_id_target_mean = all_data.groupby('item_id')['target'].mean()

# Вычисляем количество объектов для каждого item_id
item_id_count = all_data.groupby('item_id')['target'].transform('count')

# Применяем формулу сглаживания
smoothed_target = (item_id_target_mean[item_id_count > 0] * item_id_count[item_id_count > 0] + globalmean * alpha) / (item_id_count[item_id_count > 0] + alpha)

# Заполняем значения в новом столбце
all_data['smoothed_target'] = all_data['item_id'].map(smoothed_target).fillna(globalmean)

# Выводим результат
print(all_data[['item_id', 'target', 'smoothed_target']].head())

# Вычисляем коэффициент корреляции между целевой переменной и сглаженным признаком
corr = np.corrcoef(all_data['target'].values, all_data['smoothed_target'].values)[0][1]
print(corr)

        item_id  target  smoothed_target
139255       19     0.0         0.105665
141495       27     0.0         0.202869
144968       28     0.0         0.153338
142661       29     0.0         0.072900
138947       32     6.0         1.177759
0.4821354242646004


Ожидаемый ответ 0.4818

### Expanding mean схема (**1.25 балла**)

Необходимо реализовать *expanding mean* схему. Ее суть заключается в том, чтобы пройти по отсортированному в определенном порядке датасету (датасет сортируется в самом начале задания) и для подсчета счетчика для строки $m$ использовать строки от $0$ до $m-1$. Вам будет необходимо воспользоваться pandas функциями [`cumsum`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.cumsum.html) и [`cumcount`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.cumcount.html).

In [38]:
# Сортируем данные по номеру блока, shop_id и item_id
all_data.sort_values(['date_block_num', 'shop_id', 'item_id'], inplace=True)

# Создаем вспомогательные переменные для вычисления накопленных сумм и количества строк
# Cumulative sum по целевой переменной 'target' для каждого 'item_id'
cumsum = all_data.groupby('item_id')['target'].cumsum() - all_data['target']

# Cumulative count для количества строк до текущей
cumcnt = all_data.groupby('item_id').cumcount()

# Вычисляем expanding mean (накопленное среднее) для каждой строки
all_data['item_target_enc'] = cumsum / cumcnt

# Заполняем возможные NaN значения средним по всей выборке
all_data['item_target_enc'].fillna(0.3343, inplace=True)

# Вычисляем корреляцию между целевой переменной и закодированным признаком
encoded_feature = all_data['item_target_enc'].values
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]

print(corr)

<ipython-input-38-5ae5652861d4>:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  all_data['item_target_enc'].fillna(0.3343, inplace=True)


0.5025245211081697


Ожидаемый ответ 0.5025